In [1]:
import numpy as np
import pandas as pd
import pickle
import progressbar

In [ ]:
!head ../data/u_id.tsv

In [ ]:
!head ../data/l_followers.tsv

In [2]:
uidf = '../data/u_id.tsv'
follf = '../data/l_friends.tsv'

In [3]:
def read_uids():
    fp = open(uidf)
    uids = []
    for i, line in progressbar.progressbar(enumerate(fp)):
        if line != '\n':
            uids.append(int(line))
    fp.close()
    return np.array(uids)

In [ ]:
uids = read_uids()
suids = pd.Series(uids)

In [4]:
def uididx(uid):
    if not (suids==uid).any():
        return None
    return suids[suids==uid].index[0]

In [5]:
chunksize = 1000
chunk_folder = '../data/chunks/'

In [9]:
pickle.dump(suids, open(chunk_folder + 'UIDS.pck', 'wb'))

In [6]:
suids = pickle.load(open(chunk_folder + 'UIDS.pck', 'rb'))

In [ ]:
lens = []
for i in progressbar.progressbar(range(1587616)):
    lens.append(len(read_followings(i)))
slens = pd.Series(lens)


In [28]:
pickle.dump(slens, open(chunk_folder + 'SLENS.pck', 'wb'))

In [30]:
slens = pickle.load(open(chunk_folder + 'SLENS.pck', 'rb'))

In [48]:
blacklist = slens[slens > 100].index
len(blacklist)

683937

In [10]:
fp = open(follf)
chunk = []
for i, line in progressbar.progressbar(enumerate(fp)):
    if line != '\n':
        chunk.append(np.array(line.split(',')).astype(int))
    else:
        chunk.append(np.array([]))
    #if len(chunk[-1]) > 0 and chunk[-1][0] == 2586398862:
    #    print('here', i)
    if i > 0 and len(chunk) == chunksize:
        fileidx = i // chunksize
        pickle.dump(chunk, open(chunk_folder + 'friends_' + str(fileidx) + '.pck', 'wb'))
        chunk = []
        
fileidx = (i // chunksize) + 1
pickle.dump(chunk, open(chunk_folder + 'friends_' + str(fileidx) + '.pck', 'wb'))

fp.close()

| |        #                                    | 1587617 Elapsed Time: 0:14:33


In [49]:
#1587617

In [50]:
from functools import lru_cache

@lru_cache(maxsize=32)
def get_chunkfile(file):
    return pickle.load(open(file, 'rb'))

In [51]:
def read_followings(idx):
    if idx == None or idx in blacklist:
        return np.array([])
    fileidx = (idx // chunksize)
    infileidx = idx - ((fileidx) * chunksize)
    file = chunk_folder + 'friends_' + str(fileidx) + '.pck'
    chunk = get_chunkfile(file)
    return chunk[infileidx]

In [52]:
def read_followings_slow(idx):
    fp = open(follf)
    for i, line in enumerate(fp):
        if i == idx:
            if line == '\n':
                friends =  np.array([])
            else:
                friends = np.array(line.split(',')).astype(int)
            fp.close()
            return friends

i = 200
f2 = read_followings_slow(i)
f1 = read_followings(i) 
(f1==f2).all()

In [56]:
for i in range(1000):
    fls = read_followings(i)
    if len(fls) > 50  and len(fls) < 150:
        print(i)
        break

8


In [57]:
import networkx as nx
import itertools

In [58]:
def recurse_fls(curuid, depth=0, maxdepth=2):
    idx = uididx(curuid)
    fls = read_followings(idx)
    add_to_graph(G, curuid, fls, distance=depth)
    if depth+1 < maxdepth:
        for uid in fls:
            recurse_fls(uid, depth+1, maxdepth)        

In [59]:
def add_to_graph(G, uid, fls, **kwargs):
    G.add_nodes_from(fls.tolist(), **kwargs)
    G.add_edges_from(zip(itertools.cycle([int(uid)]), fls.tolist()))

In [78]:
G = nx.Graph()
recurse_fls(suids[19], maxdepth=5)

In [79]:
nx.set_node_attributes(G, 2.5, 'x')
nx.set_node_attributes(G, 2.5, 'y')
nx.set_node_attributes(G, '-', 'name')

In [80]:
jsongraph = nx.node_link_data(G)

In [82]:
len(jsongraph['nodes'])

2857

In [83]:
import json

In [84]:
jsongraph['_comment_data_source'] = 'https://zenodo.org/record/1135171#.XNc_4aZCR25'

In [85]:
from networkx.readwrite import json_graph

In [86]:
mapping = dict()
nodes = jsongraph['nodes']
for i in range(len(nodes)):
    mapping[nodes[i]['id']] = i

for link in jsongraph['links']:
    link['source'] = mapping[link['source']]
    link['target'] = mapping[link['target']]

In [87]:
with open('../hacktival2019/webapp/js/graph.json', 'w') as outfile:
    outfile.write(json.dumps(jsongraph))

In [ ]:
jsongraph

In [89]:
#for node in jsongraph['nodes']:
#    print(node['distance'])
    #print(link)

nx.draw_networkx(G)